In [1]:
#Figure 6 graph plotting 
#v1.0 version, this is the first consolidation of analysis done so far
#Requires the environment packages found in 23pubenv

In [2]:
import tifffile
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from skimage import filters, morphology, measure, segmentation
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
from scipy.signal import find_peaks
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

In [3]:
def im_stack_thresh(im_stack): 
    oats = []

    for i in range(len(im_stack)):
        oats.append(filters.threshold_otsu(im_stack[i]))

    return np.mean(oats)

In [53]:
#Get all the folders needed:
folderpath = Path.cwd() / 'accumulation-stills' / 'raw-accumulation-files'

#Get a list of all the images in each folder: 
img_paths = sorted(folderpath.glob('*.tif'), key= lambda x: x.stem.split('_'))


In [54]:
img_paths

[PosixPath('/Users/echo/Documents/Lab Stuff/Publication/23 Protofigures/Fig6 - Accumulation/accumulation-stills/raw-accumulation-files/10m-w1-p2-t3.tif'),
 PosixPath('/Users/echo/Documents/Lab Stuff/Publication/23 Protofigures/Fig6 - Accumulation/accumulation-stills/raw-accumulation-files/3m-w3-p1-t2.tif'),
 PosixPath('/Users/echo/Documents/Lab Stuff/Publication/23 Protofigures/Fig6 - Accumulation/accumulation-stills/raw-accumulation-files/6m-w3-p3-t4.tif')]

In [55]:
file_num = 1

im_file = tifffile.imread(img_paths[file_num])
im_name = img_paths[file_num].stem

#find in focus frame of z-stack: 

stds = []

for frame in im_file: 
    stds.append(np.std(frame))
    
in_focus_frame = im_file[np.argmax(stds)]

In [51]:
export_location = folderpath.parent / 'wt healthy'

#All images with same stretching: 800-3500
vmin_set = 350
vmax_set = 1000
border_color_core = '#EE362B'
border_color_cells = '#F5ABB9'
border_width = 10


In [52]:
#Generating stills with boundry of accumulation overlay on nuclear imaging:

#Get the final core centroid first:

threshold = im_stack_thresh(in_focus_frame)

frame = in_focus_frame

#Threshold core by otsu thresh for stack, then remove all single cells etc
bin_im = frame > threshold
core_adj = morphology.erosion(bin_im)
core_adj = morphology.remove_small_objects(core_adj, 20)
core_adj = morphology.dilation(core_adj)
boundary_core = measure.find_contours(core_adj)

#Plotting
fig, ax = plt.subplots(figsize=(10,10))
plt.imshow(frame, cmap='Greys_r', vmin=vmin_set,vmax=vmax_set)
plt.axis('off')
for contour in boundary_core:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=4, fillstyle='full', color=border_color_core)

    
#boundary_cells = measure.find_contours(cell_bin)
#for contour in boundary_cells:
#    ax.plot(contour[:, 1], contour[:, 0], linewidth=1, color=border_color_cells)
plt.gca().invert_yaxis()

scalebar = AnchoredSizeBar(ax.transData,
                           100/2.2, '',  'upper right', 
                           pad=1,
                           color='white',
                           frameon=False,
                           size_vertical=5)

ax.add_artist(scalebar)
plt.savefig(export_location / Path('{}-still.png'.format(im_name)), dpi=300, bbox_inches='tight', pad_inches=0)
plt.close()
